# Clustering Lab

 
Based of the amazing work you did in the Movie Industry you've been recruited to the NBA! You are working as the VP of Analytics that helps support a head scout, Mr. Rooney, for the worst team in the NBA probably the Wizards. Mr. Rooney just heard about Data Science and thinks it can solve all the team's problems!!! He wants you to figure out a way to find players that are high performing but maybe not highly paid that you can steal to get the team to the playoffs! 

In this document you will work through a similar process that we did in class with the NBA data (NBA_Perf_22 and nba_salaries_22), merging them together. This is from 22-23 season, feel free to update to 2023-24 season if you want.
# Data Sources:

https://www.basketball-reference.com/leagues/NBA_2024_totals.html # reference for performance data
https://www.basketball-reference.com/contracts/players.html # reference for salary data


Details: 

- Determine a way to use clustering to estimate based on performance if 
players are under or over paid, generally. 

- Then select players you believe would be best for your team and explain why. Do so in three categories: 
    * Examples that are not good choices (3 or 4) 
    * Several options that are good choices (3 or 4)
    * Several options that could work, assuming you can't get the players in the good category (3 or 4)

- You will decide the cutoffs for each category, so you should be able to explain why you chose them.

- Provide a well commented and clean report of your findings in a separate notebook that can be presented to Mr. Rooney, keeping in mind he doesn't understand...anything. Include a rationale for variables you included in the model, details on your approach and a overview of the results with supporting visualizations. 


Hints:

- Salary is the variable you are trying to understand 
- When interpreting you might want to use graphs that include variables that are the most correlated with Salary
- You'll need to scale the variables before performing the clustering
- Be specific about why you selected the players that you did, more detail is better
- Use good coding practices, comment heavily, indent, don't use for loops unless totally necessary and create modular sections that align with some outcome. If necessary create more than one script,list/load libraries at the top and don't include libraries that aren't used. 
- Be careful for non-traditional characters in the players names, certain graphs won't work when these characters are included.


In [23]:
# Importing libraries

import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

# Reading the CSV files (used copilot to help handle special characters that pandas could not read)
nba_performance_22 = pd.read_csv("/workspaces/DS-3021/data/NBA_Perf_22.csv", encoding="ISO-8859-1")
nba_salaries_22 = pd.read_csv("/workspaces/DS-3021/data/nba_salaries_22.csv")

#Drop variables that will not be needed or are duplicates

print(nba_performance_22.info()) # Printing the variables from the dataset

# Dropping unnecessary columns

nba_performance = nba_performance_22.drop(columns = ['Player', 'Tm', 'Pos', 'FGA', 'FG', '3P', '3PA', '2P', '2PA', 'FT', 'FTA'])

# Dropping duplicate variables 

nba_performance = nba_performance.drop_duplicates()

nba_salaries = nba_salaries_22.drop_duplicates()

# Merging the datasets

nba_data = pd.merge(nba_performance_22[['Player']].join(nba_performance), nba_salaries_22, on = 'Player', how =  'inner')

# Removing $ from the Salary column

nba_data['Salary'] = nba_data['Salary'].replace('[\$,]', '', regex = True)

nba_data.head() # Printing the first five rows of the merged dataframe



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  812 non-null    object 
 1   Pos     812 non-null    object 
 2   Age     812 non-null    int64  
 3   Tm      812 non-null    object 
 4   G       812 non-null    int64  
 5   GS      812 non-null    int64  
 6   MP      812 non-null    float64
 7   FG      812 non-null    float64
 8   FGA     812 non-null    float64
 9   FG%     797 non-null    float64
 10  3P      812 non-null    float64
 11  3PA     812 non-null    float64
 12  3P%     740 non-null    float64
 13  2P      812 non-null    float64
 14  2PA     812 non-null    float64
 15  2P%     784 non-null    float64
 16  eFG%    797 non-null    float64
 17  FT      812 non-null    float64
 18  FTA     812 non-null    float64
 19  FT%     715 non-null    float64
 20  ORB     812 non-null    float64
 21  DRB     812 non-null    float64
 22  TR

<>:37: SyntaxWarning: invalid escape sequence '\$'
<>:37: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_571/1096488748.py:37: SyntaxWarning: invalid escape sequence '\$'
  nba_data['Salary'] = nba_data['Salary'].replace('[\$,]', '', regex = True)


,Player,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Salary
0,Precious Achiuwa,22.0,73.0,28.0,23.6,0.439,0.359,0.468,0.486,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1,2840160
1,Steven Adams,28.0,76.0,75.0,26.3,0.547,0.000,0.548,0.547,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9,17926829
2,Bam Adebayo,24.0,56.0,56.0,32.6,0.557,0.000,0.562,0.557,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1,30351780
3,Santi Aldama,21.0,32.0,0.0,11.3,0.402,0.125,0.560,0.424,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1,2094120
4,Nickeil Alexander-Walker,23.0,65.0,21.0,22.6,0.372,0.311,0.433,0.449,0.743,0.6,2.3,2.9,2.4,0.7,0.4,1.4,1.6,10.6,5009633


In [ ]:
#Run the clustering algo with your best guess for K

In [ ]:
#View the results

In [ ]:
#Create a visualization of the results with 2 or 3 variables that you think will best
#differentiate the clusters

In [ ]:
#Evaluate the quality of the clustering using total variance explained and silhouette scores

In [ ]:
#Determine the ideal number of clusters using the elbow method and the silhouette coefficient

In [ ]:
#Visualize the results of the elbow method

In [ ]:
#Use the recommended number of cluster (assuming it's different) to retrain your model and visualize the results

In [ ]:
#Once again evaluate the quality of the clustering using total variance explained and silhouette scores

In [ ]:
#Use the model to select players for Mr. Rooney to consider

#Write up the results in a separate notebook with supporting visualizations and 
an overview of how and why you made the choices you did. This should be at least 
500 words and should be written for a non-technical audience.